# SRS Conversion to Database Comparison

This report compares the raw conversion code output to the converted counts stored in the database. It then highlights any differences.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.engine import Engine
from pathlib import Path
import os
import warnings

scratch_space = Path(os.getenv("OUTPUT_PIPELINE_DIR"))
state = os.getenv("INPUT_STATE")
converted_path = scratch_space / "srs"/"extracts"
year=os.getenv("DATA_YEAR")
(scratch_space / "srs" / "QC_output_files").mkdir(exist_ok=True,parents=True)

nibrs = Path("nibrs-estimation-pipeline")
def connect_to_database() -> Engine:
    """This function opens a connection to the database."""
    user = os.getenv("PGUSER")
    password = os.getenv("PGPASSWORD")
    host = os.getenv("PGHOST")
    port = os.getenv("PGPORT")
    dbname = os.getenv("PGDATABASE")
    # --- Create connection
    engine_database = create_engine(
        f"postgresql://{user}:{password}@{host}:{port}/{dbname}"
    )
    return engine_database
engine_database = connect_to_database()

srs_col_dict = [
    'MURDER',
    'MANSLAUGHTER',
    'RAPE',
    'FORCE RAPE',
    'ATMPTD RAPE',
    'ROBBERY',
    'GUN ROBBERY',
    'KNIFE ROBBERY',
    'OTHER WEAPON ROBBERY',
    'STRONG ARM ROBBERY',
    'ASSAULT',
    'GUN ASSAULT',
    'KNIFE ASSAULT',
    'OTHER WEAPON ASSAULT',
    'HAND/FEET ASSAULT',
    'SIMPLE ASSAULT',
    'BURGLARY',
    'BURGLARY FORCE ENTRY',
    'BURGLARY ENTRY-NO FORCE',
    'ATTEMPTED BURGLARY',
    'LARCENY',
    'VEHICLE THEFT',
    'AUTO THEFT',
    'TRUCK/BUS THEFT',
    'OTHER VEHICLE THEFT',
    'ALL FIELDS',
]

month_col_dict = {
    70:"JAN",
    188:"FEB",
    306:"MAR",
    424:"APR",
    542:"MAY",
    660:"JUN",
    778:"JUL",
    896:"AUG",
    1014:"SEP",
    1132:"OCT",
    1250:"NOV",
    1368:"DEC"
}


# derive the full list of v variables we need
all_dict = {}
for month in month_col_dict.keys():
    for i in range(len(srs_col_dict)):
        all_dict[f"v{month + i}"] = month_col_dict[month] + " " + srs_col_dict[i]

# these are fields which are aggregates of other fields, so we drop them when comparing
total_fields = ["ALL FIELDS","ROBBERY","RAPE","ASSAULT","BURGLARY","VEHICLE THEFT"]
invalid_fields = []
for field in total_fields:
    invalid_fields += [f"{month} {field}" for month in month_col_dict.values()]
    
    
breakdown_dict = {
    "Assault - Firearm":'GUN ASSAULT',
    "Assault - Hands, Fists, Feet":'HAND/FEET ASSAULT',
    "Assault - Knife or Cutting Instrument":'KNIFE ASSAULT',
    "Assault - Other Dangerous Weapon":'OTHER WEAPON ASSAULT',
    "Burglary - Attempted Forcible Entry":'ATTEMPTED BURGLARY',
    "Burglary - Forcible Entry":'BURGLARY FORCE ENTRY',
    "Burglary - No Force":'BURGLARY ENTRY-NO FORCE',
    "Auto Theft":'AUTO THEFT',
    "Other Vehicle Theft":'OTHER VEHICLE THEFT',
    "Truck and Bus Theft":'TRUCK/BUS THEFT',
    "Murder and Nonnegligent Homicide":'MURDER',
    "Attempted Rape":'ATMPTD RAPE',
    "Rape":'FORCE RAPE',
    "Robbery - Firearm":'GUN ROBBERY',
    "Robbery - Hands, Fists, Feet":'STRONG ARM ROBBERY',
    "Robbery - Knife or Cutting Instrument":'KNIFE ROBBERY',
    "Robbery - Other Dangerous Weapon":'OTHER WEAPON ROBBERY',
    "Simple Assault":'SIMPLE ASSAULT',
    "Manslaughter by Negligence":'MANSLAUGHTER',
    'Larceny - Theft (Not Specified)':"LARCENY",
}

month_list = list(month_col_dict.values())

In [ ]:
# get the output of step 2 of the SRS conversion

with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    df_conversion_aggregated = pd.read_csv(converted_path / f"SRS_{year}_{state}.csv").drop(
        columns=["ori"]
    ).rename(columns={"legacy_ori":"ori"}).sort_values(by=["ori"]).reset_index(drop=True)

    # get the SRS external file
    df_srs_fromdb = pd.read_csv(
            scratch_space /
            "initial_tasks_output" /
            f"UCR_SRS_{year}_clean_reta_mm_selected_vars.csv",
            parse_dates=["NIBRS_START_DATE_UNIV"]
    ).rename(columns={"ORI":"ori"})

# get the nibrs start dates for each ori
unique_ori_unique = df_srs_fromdb[["ori","NIBRS_START_DATE_UNIV"]].drop_duplicates()

# subset to just the one state we have the conversion file for
df_srs_fromdb = df_srs_fromdb.loc[df_srs_fromdb["ori"].isin(
    df_conversion_aggregated["ori"]
)].sort_values(by=["ori"]).reset_index(drop=True)[df_conversion_aggregated.columns]

# rename columns from V form to text
df_conversion_aggregated = df_conversion_aggregated.rename(
    columns=all_dict
).melt(id_vars=["ori"])
df_srs_fromdb = df_srs_fromdb.rename(
    columns=all_dict
).melt(id_vars=["ori"])
both_merged = df_conversion_aggregated.merge(df_srs_fromdb,on=["ori","variable"],suffixes=["_rti","_cjis"])


# drop ori+variable combos for variables where the start date was both the year in question 
# and from a month after that variable's month
def get_start_month(d):
    if d == -1:
        return d
    if d.year > int(year):
        # if the start year was after this year then no months are valid
        return 13
    elif d.year < int(year):
        # if the start year was before this year then all months are valid
        return -1
    else:
        return d.month

unique_ori_unique["NIBRS_START_MONTH"] = unique_ori_unique["NIBRS_START_DATE_UNIV"].fillna(-1).apply(get_start_month)
month_num_dict = {month_list[i]:i+1 for i in range(len(month_list))}
both_merged["variable_month"] = both_merged["variable"].apply(lambda x: month_num_dict[x.split(" ")[0]])
both_merged = both_merged.merge(unique_ori_unique[["ori","NIBRS_START_MONTH","NIBRS_START_DATE_UNIV"]],on="ori",how="left")
both_merged = both_merged.loc[both_merged["variable_month"] >= both_merged["NIBRS_START_MONTH"]].drop(
    columns=["variable_month","NIBRS_START_MONTH"]
).copy()

In [ ]:
print(f"In this report we are looking at counts for the state of {state} for {year}")

## Part 1: Overall Differences

Below are the overall differences in the Return A SRS offense counts for NIBRS agencies between the rti-converted counts and the cjis-converted table. 

First, let's see which agencies in the state had differences in their overall counts for the year.

In [ ]:
ori_level_diffs = both_merged.loc[both_merged["variable"].str.endswith("ALL FIELDS")].groupby(["ori"])[["value_rti","value_cjis"]].sum()
ori_level_diffs = ori_level_diffs.loc[ori_level_diffs["value_rti"] != ori_level_diffs["value_cjis"]].copy()
ori_level_diffs["diff"] = ori_level_diffs["value_rti"] - ori_level_diffs["value_cjis"]
with pd.option_context('display.max_rows', None):
    display(ori_level_diffs.sort_values(by=["diff"],ascending=False))

Next, let's break these differences down by variable and month.

In [ ]:
merged_index = both_merged.groupby(["variable"])[["value_rti","value_cjis"]].sum()

all_indexes = []
for month in month_num_dict.keys():
    all_indexes += [f"{month} {x}" for x in srs_col_dict]

rows_to_drop = [i for i in invalid_fields if i in merged_index.index]
merged_index = merged_index.reindex(
    [r for r in merged_index.index if r not in rows_to_drop]
)
# sort the index by month and varible and then add that sort order as an index we can use to re-sort later
merged_index = merged_index.reindex(
    [r for r in all_indexes if r in merged_index.index]
).reset_index().reset_index().set_index("variable")
merged_index["diff"] = merged_index["value_rti"] - merged_index["value_cjis"]
merged_index = merged_index.loc[merged_index["diff"] != 0]
with pd.option_context('display.max_rows', None):
    display(merged_index.sort_values(by=["diff","index"],ascending=[False,True]).drop(columns=["index"]))

## Part 2: Specific Agency Differences

In this section we identify the specific sources of differences seen in the aggregate totals.

First, below we see the agency-level differences for each variable. 

In [ ]:
diffs = both_merged.loc[both_merged["value_rti"] != both_merged["value_cjis"]]
# ignore total fields since we have the detailed categories
diffs = diffs.loc[~diffs["variable"].isin(invalid_fields)].rename(columns={
    "value_rti":"count_rti",
    "value_cjis":"count_cjis"
})
# display the places where we see different counts
with pd.option_context('display.max_rows', None):
    display(diffs.reset_index(drop=True)[["ori","NIBRS_START_DATE_UNIV","variable","count_rti","count_cjis"]])

In [ ]:
if len(diffs) > 0:

    df_conversion_detailed = pd.read_csv(
        converted_path / 
        f"raw_SRS_incidents_{year}_{state}.csv"
    )
    df_conversion_detailed["offense"] = df_conversion_detailed["der_offense_month"].apply(lambda x: all_dict[x])
    df_conversion_detailed = df_conversion_detailed[["legacy_ori","incident_id","offense","counts"]].rename(
        columns={"offense":"variable","counts":"count_rti"}
    )
    df_conversion_detailed["match"] = df_conversion_detailed["legacy_ori"] + "_" + df_conversion_detailed["variable"]

    ori_list = "('"+"','".join(diffs['ori'].unique().tolist())+"')"
    specific_incident_raw = f"""
        select
            ra.legacy_ori,
            lso.offense_name,
            lso.breakdown_name,
            data_month,
            smo.nibrs_incident_id,
            smo.actual_count from ucr_prd.form_month fm
        join ucr_prd.sum_month_offense smo using(form_month_id)
        join ucr_prd.ref_agency ra using (agency_id)
        join ucr_prd.lkup_srs_offense lso using (breakdown_id)
        WHERE
        data_year = {year} and ra.legacy_ori in {ori_list}
    """
    specific_incident_df = pd.read_sql(specific_incident_raw, engine_database).drop_duplicates()
    specific_incident_df["offense"] = specific_incident_df["breakdown_name"].apply(lambda x: breakdown_dict[x] if x in breakdown_dict else None)
    specific_incident_df.dropna(subset=["offense"],inplace=True)
    specific_incident_df["offense"] = specific_incident_df["data_month"].apply(lambda m: month_list[m - 1]) + " " + specific_incident_df["offense"]
    specific_incident_df = specific_incident_df[["legacy_ori","nibrs_incident_id","offense","actual_count"]].rename(columns={
        "nibrs_incident_id":"incident_id","actual_count":"count_cjis","offense":"variable"
    })
    specific_incident_df["match"] = specific_incident_df["legacy_ori"] + "_" + specific_incident_df["variable"]

In [ ]:

if len(diffs) > 0:
    print("Finally, let's split these differences out by specific NIBRS incident ID.")
    # for pairs of diffs, get the rows from both datasets which are around each
    # then merge the two datasets on incident ID
    diffs["match"] = diffs["ori"] + "_" + diffs["variable"]
    converted_incidents = df_conversion_detailed.loc[df_conversion_detailed["match"].isin(diffs["match"])].drop(columns=["match"])
    db_incidents = specific_incident_df.loc[specific_incident_df["match"].isin(diffs["match"])].drop(columns=["match"])

    converted_incidents["incident_id"] = converted_incidents["incident_id"].astype(float)
    db_incidents["incident_id"] = db_incidents["incident_id"].astype(float)

    both_groups = converted_incidents.merge(db_incidents,on=["legacy_ori","incident_id","variable"],how="outer")
    both_groups["count_rti"] = both_groups["count_rti"].fillna(0)
    both_groups["count_cjis"] = both_groups["count_cjis"].fillna(0)

    both_groups = both_groups.loc[both_groups["count_rti"] != both_groups["count_cjis"]]
    both_groups = both_groups.sort_values(by=["variable","legacy_ori"])[
        ['legacy_ori', 'incident_id', 'variable','count_rti', 'count_cjis']
    ].reset_index(drop=True)
    both_groups.to_csv(scratch_space / "srs" / "QC_output_files" / f"incident_level_differences_{state}_{year}.csv",index=False)

    with pd.option_context('display.max_rows', None):
        display(both_groups)